# 기본 설정

## 필요 모듈 import. 차트의 한글 설정 등

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 40
plt.rcParams['figure.figsize'] = [40, 20] # 전역으로 차트의 크기 설정
def setTick():
    plt.tick_params(colors='white', which='both', labelsize=20)

plt.style.use('default')
    
fontdictHead={'fontsize': 50, 'color': 'yellow'}
fontdictBody={'fontsize': 30, 'color': 'white'}

# if mac
def ifMac() :
    plt.rc('font', family='AppleGothic')
    plt.rcParams['axes.unicode_minus'] = False    

# ifMac()

plt.rc("font", family="Malgun Gothic")
plt.rc('axes', unicode_minus=False)
from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")
plt.title('한글폰트 설정')
plt.plot([-4, -6, 1, 2, 0, 3])


## 상가업소정보_201912_01.csv 파일을 불러와서 df에 저장

In [ ]:
df = pd.read_csv('./Merchant/상가업소정보_201912_01.csv', sep='|')
pd.options.display.max_columns = 39

## n변수에 각 컬럼의 결측치 갯수를 모두 합한 결과를 담습니다

In [ ]:
n = df.isna().sum()

## n 변수에 담긴 값을 정렬해서 상위 9개만 not_use변수에 담습니다.

In [ ]:
not_use = n.sort_values(ascending=False).head(9)

## 저장된 결측치 데이터에서 결측치 합계를 제외하고 행제목(인덱스)들만 따로 not_use_col 변수에 저장

In [ ]:
not_use_col = not_use.index

## 변수의값들을 갖고 df에서 컬럼들을 제거

In [ ]:
print(df.shape)
df = df.drop(not_use_col, axis=1)
print(df.shape)

## 코드|번호가 포함된 컬럼명을 처리하기 위해 cols라는 변수에컬럼이름들을 담습니다.

In [ ]:
cols = df.columns

## "코드", "번호"가 들어가는 컬럼명만 drop으로 데이터프레임에서 제거

In [ ]:
del_cols = cols[cols.str.contains('코드|번호')]

print(df.shape)
df = df.drop(del_cols, axis=1)
print(df.shape)

df.head()

## 추가로 "대지구분명", "지번본번지", "지번주소", "도로명", "건물본번지"도 삭제합니다

In [ ]:
print(df.shape)
df = df.drop(['대지구분명', '지번본번지', '지번주소', '도로명', '건물본번지'], axis=1)
print(df.shape)

## 서울데이터만 필터링합니다.

In [ ]:
df_seoul = df[df['시도명'] == '서울특별시']

## "seoul_open_store.csv" 라는 이름으로 저장합니다.

In [ ]:
df_seoul.to_csv('seoul_open_store.csv', index=False)

#  베스킨라빈스, 던킨도너츠 위치 분석
## 데이터 정제
- 여기에서는 베스킨라빈스와 던킨도너츠 상호를 가져와서 실습합니다.
- 위에서 pandas의 str.contains를 활용해봅니다.
- 상호명에서 브랜드명을 추출합니다.
- 대소문자가 섞여 있을 수도 있기 때문에 대소문자를 변환해 줍니다.
- 오타를 방지하기 위해 베스킨라빈스의 영문명은 baskinrobbins, 던킨도너츠는 dunkindonuts입니다.

In [ ]:
df_seoul.head()

### 영문 소문자로 변경 후 저장

In [ ]:
df_seoul['상호명_소문자'] = df_seoul['상호명'].str.lower()

In [ ]:
df_seoul.head()

### 베스킨라빈스 조회

In [ ]:
df_seoul.loc[df_seoul['상호명_소문자'].str.contains('배스킨라빈스|베스킨라빈스|baskinrobbins'), '상호명_소문자'].shape

### 던킨도너츠 조회

In [ ]:
df_seoul.loc[df_seoul['상호명_소문자'].str.contains('던킨|dunkin'), '상호명_소문자'].shape

### 베스킨라빈스 + 던킨도너츠로 필터링하고, df_31에 저장합니다.

In [ ]:
df_31 = df_seoul[df_seoul['상호명_소문자'].str.contains('배스킨라빈스|베스킨라빈스|baskinrobbins|던킨|dunkin')]
df_31.shape

In [ ]:
df_31.head()

### '브랜드명'이라는 새로운 컬럼 추가

In [ ]:
# 상호명_소문자를 통해 새로운 컬럼 '브랜드명'컬럼에 '배스킨라빈스'값 추가
df_31.loc[df_31['상호명_소문자'].str.contains('배스킨라빈스|베스킨라빈스|baskinrobbins'), '브랜드명'] = '배스킨라빈스'

In [ ]:
# 브랜드명의 결측치(빈칸)에 던킨도너츠를 채웁니다.
df_31['브랜드명'] = df_31['브랜드명'].fillna('던킨도너츠')

In [ ]:
df_31[['상호명', '브랜드명']].head()

In [ ]:
# 데이터가 제대로 모아졌는지 확인합니다.
# "상권업종대분류명"을 value_counts로 빈도수를 체크합니다.
df_31['상권업종대분류명'].value_counts()

In [ ]:
# '상권업종대분류명'컬럼에서 isin 기능을 사용해서 '소매', '생활서비스'인 데이터만 검색해봅니다.
df_31[df_31['상권업종대분류명'].isin(['소매', '생활서비스'])]

### 소매, 생활서비스가 아닌것만 필터링

In [ ]:
# & : and , | : or , ~ : not - not연산자를 통해 소매, 생활서비스가 아닌것만 필터링합니다.
df_31 = df_31[~df_31['상권업종대분류명'].isin(['소매', '생활서비스'])]

In [ ]:
df_31.shape

## 시각화

### plot.barh

In [ ]:
# 브랜드명으로 각 브랜드의 갯수를 표현
df_31['브랜드명'].value_counts().plot.barh()

### plot barh normalize

In [ ]:
# 브랜드명의 value_counts값을 비율로 표시하고 이를 차트에 표현합니다.
df_31['브랜드명'].value_counts(normalize=True).plot.barh()

### countplot

In [ ]:
# seaborn과 countplot을 이용한 그래프
brand_count = df_31['브랜드명'].value_counts()
g = sns.countplot(data = df_31, x='브랜드명')

# x, y좌표에 brand_count 값을 표시합니다.
for i, val in enumerate(brand_count.index) :
    g.text(x=i, y=brand_count[i]+20, s=brand_count[i])

### countplot

In [ ]:
# 시군구명으로 빈도수를 세고, 브랜드명으로 다르게 표현하는 countplot을 그립니다.
plt.figure(figsize=(15, 4))
g = sns.countplot(data=df_31, x='시군구명', hue='브랜드명')

### scatterplot

In [ ]:
# seaborn의 scatterplot으로 hue에 브랜드명을 지정해서 시각화합니다.
sns.scatterplot(data=df_31, x='경도', y='위도', hue='브랜드명')

### folium

In [ ]:
# Lat : 37.495593 Long : 127.039032 tooltip: '던킨도너츠'의 값으로
# folium 맵에 마킹하세요

##### import

In [ ]:
import folium

### 중앙 기준으로 지도표출 1: tooltip하나 생성

In [246]:
# 중앙값 = 각각 쳥균값
centerLat, centerLong = df_31['위도'].mean(), df_31['경도'].mean()

# 임의의 위경도와 툴팁설정
lat, long, tooltip = 37.495593,127.039032, '던킨도너츠'

# 지도표출
m = folium.Map(location=[centerLat, centerLong], zoom_start=12)
folium.Marker([lat, long], tooltip=tooltip).add_to(m)
m

### 중앙 기준으로 지도표출 2: 모든 매장 출력
- df_31의 위도값들과 경도값들을 이용하여 모두마킹
- tooltip값은 브랜드명-도로명주소로 설정합니다.
- zoom_start는 12 또는 11 중 적절히 설정하세요
- 던킨도너츠는 빨간색, 베스킨라빈스는 파란색으로 마킹하세요.

In [253]:
# 중앙값 = 각각 쳥균값
centerLat, centerLong = df_31['위도'].mean(), df_31['경도'].mean()
m = folium.Map(location=[centerLat, centerLong], zoom_start=12)

# 모든 값 순회
for i in df_31.index[:] :
    tooltip = df_31.loc[i, '상호명'] + ' - (' + df_31.loc[i, '도로명주소'] + ')'
    lat = df_31.loc[i, '위도']
    long = df_31.loc[i, '경도']
        
    folium.Marker([lat, long], tooltip=tooltip, ).add_to(m)
m

### 중앙 기준으로 지도표출 3: 모든 매장 마커로 출력

In [258]:
# 중앙값 = 각각 쳥균값
centerLat, centerLong = df_31['위도'].mean(), df_31['경도'].mean()
m = folium.Map(location=[centerLat, centerLong], zoom_start=12)

# 모든 값 순회
for i in df_31.index[:] :
    tooltip = df_31.loc[i, '상호명'] + ' - (' + df_31.loc[i, '도로명주소'] + ')'
    lat = df_31.loc[i, '위도']
    long = df_31.loc[i, '경도']
    
    if df_31.loc[i, '브랜드명'] == '배스킨라빈스' :
        icon_color = 'blue'
    else :
        icon_color = 'red'
        
    folium.CircleMarker([lat, long], tooltip=tooltip, color=icon_color, radius=3).add_to(m)
m